# РК №2 ТМО
## Секретов ИУ5-62
## Вариант 15

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [3]:
data_raw=pd.read_csv('states_all_extended.csv',sep=",")

In [4]:
data_raw.head()


,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,G08_HI_A_READING,G08_HI_A_MATHEMATICS,G08_AS_A_READING,G08_AS_A_MATHEMATICS,G08_AM_A_READING,G08_AM_A_MATHEMATICS,G08_HP_A_READING,G08_HP_A_MATHEMATICS,G08_TR_A_READING,G08_TR_A_MATHEMATICS
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data=data_raw[['STATE','YEAR','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE','INSTRUCTION_EXPENDITURE']]

In [6]:
data.head()


,STATE,YEAR,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE
0,ALABAMA,1992,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0
1,ALASKA,1992,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0
2,ARIZONA,1992,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0
3,ARKANSAS,1992,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0
4,CALIFORNIA,1992,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0


In [7]:
data.isnull().sum()

STATE                        0
YEAR                         0
TOTAL_REVENUE              440
FEDERAL_REVENUE            440
STATE_REVENUE              440
LOCAL_REVENUE              440
TOTAL_EXPENDITURE          440
INSTRUCTION_EXPENDITURE    440
dtype: int64

In [8]:
data=data.dropna(axis=0,how='any')
data.shape

(1275, 8)

In [9]:
data.dtypes

STATE                       object
YEAR                         int64
TOTAL_REVENUE              float64
FEDERAL_REVENUE            float64
STATE_REVENUE              float64
LOCAL_REVENUE              float64
TOTAL_EXPENDITURE          float64
INSTRUCTION_EXPENDITURE    float64
dtype: object

In [10]:
label_encoder = LabelEncoder()

data['STATE'] = label_encoder.fit_transform(data['STATE'])

Разделим на обучающую и тестовую выборки

In [11]:
data_X_train, data_X_test, data_y_train, data_y_test = train_test_split(
    data, data['STATE'], test_size=0.5, random_state=1)

In [12]:
del data_X_train['STATE']
del data_X_test['STATE']

In [13]:
scaler = StandardScaler()
data_X_train_scaled = scaler.fit_transform(data_X_train)
data_X_test_scaled = scaler.transform(data_X_test)

In [14]:
data_y_test


920      2
817      1
443     35
701     38
108      6
        ..
45      45
1009    40
925      7
638     26
68      17
Name: STATE, Length: 638, dtype: int32

# Метод опорных векторов

In [15]:
svr = SVR()
param_grid_svr = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}
grid_search_svr = GridSearchCV(svr, param_grid_svr, cv=5, scoring='neg_mean_squared_error')
grid_search_svr.fit(data_X_train_scaled, data_y_train)
svr = grid_search_svr.best_estimator_

In [16]:
rfc = RandomForestClassifier(n_estimators=20, criterion='gini', max_depth=10)
rfc.fit(data_X_train, data_y_train)

RandomForestClassifier(max_depth=10, n_estimators=20)

# Оценка качества моделей

In [17]:
y_pred_svr = svr.predict(data_X_test_scaled)
y_pred_rfc = rfc.predict(data_X_test)

mae_svr = mean_absolute_error(data_y_test, y_pred_svr)
mse_svr = mean_squared_error(data_y_test, y_pred_svr)

mae_rfc = mean_absolute_error(data_y_test, y_pred_rfc)
mse_rfc = mean_squared_error(data_y_test, y_pred_rfc)

print(f"Метод опорных векторов: MSE = {mse_svr}, MAE = {mae_svr}")
print(f"Случайный лес: MSE = {mse_rfc}, MAE = {mae_rfc}")

Метод опорных векторов: MSE = 201.81487610980875, MAE = 11.451364516377055
Случайный лес: MSE = 172.38871473354232, MAE = 7.420062695924765


# Вывод

MSE измеряет среднюю величину квадрата ошибок между предсказанными значениями и фактическими значениями. 

MAE измеряет среднюю величину абсолютных ошибок между предсказанными значениями и фактическими значениями.

Случайный лес показывает лучшие результаты по сравнению с методом опорных векторов, хотя обе модели сработали не очень хорошо